In [27]:
import json
import os
import subprocess
from glob import glob

import pandas as pd
from pprint import pprint

DATA_DIR = './data'
DATA_TYPE = 'geo'

データのディレクトリ構造

In [2]:
%%bash
tree ../data | head

../data
└── 2017
    └── 06
        ├── 06
        │   ├── 00
        │   │   ├── 30.json
        │   │   ├── 31.json
        │   │   ├── 32.json
        │   │   ├── 33.json
        │   │   ├── 34.json


In [3]:
f = open(os.path.join(DATA_DIR, '2017/06/06/00/30.json'), 'r')
tweets = json.load(f)

geoタグなしのツイート

分あたり2, 300件ある

In [4]:
print(len(tweets))
pprint(tweets[0])

271
{'coordinates': None,
 'created_at': 'Tue Jun 06 06:30:00 +0000 2017',
 'emojis': [{'short_name': 'tired_face', 'unified': '1F62B'}],
 'entities_hashtags': [],
 'entities_urls': [],
 'geo': None,
 'id_str': '871977455724134400',
 'in_reply_to_status_id_str': None,
 'lang': 'en',
 'place': None,
 'retweeted_status_id_str': '871710739433652224',
 'source': '<a href="http://twitter.com/download/iphone" '
           'rel="nofollow">Twitter for iPhone</a>',
 'text': "RT @iveryy_: I pray to GOD my son ain't like none of you corny "
         'niggas 😫',
 'timestamp_ms': '1496730600659',
 'user_followers_count': 906,
 'user_friends_count': 362,
 'user_id_str': '68872363',
 'user_lang': 'en',
 'user_screen_name': 'jessx213',
 'user_statuses_count': 128183,
 'user_utc_offset': -14400}


geoタグ付きのツイート

In [5]:
for x in tweets:
    if x['geo'] is not None:
        break
        
geotagged_tweet = x
pprint(x)

{'coordinates': {'coordinates': [55.15189319, 25.12818472], 'type': 'Point'},
 'created_at': 'Tue Jun 06 06:30:52 +0000 2017',
 'emojis': [{'short_name': 'heart_eyes', 'unified': '1F60D'},
            {'short_name': 'sunny', 'unified': '2600'}],
 'entities_hashtags': [{'indices': [6, 12], 'text': 'dubai'}],
 'entities_urls': [{'display_url': 'instagram.com/p/BU_PZYflRKHt…',
                    'expanded_url': 'https://www.instagram.com/p/BU_PZYflRKHt3MX5iRjC1nC6Jq5IcA1wHrJOpI0/',
                    'indices': [68, 91],
                    'url': 'https://t.co/zPY9SKJXgw'}],
 'geo': {'coordinates': [25.12818472, 55.15189319], 'type': 'Point'},
 'id_str': '871977673823793153',
 'in_reply_to_status_id_str': None,
 'lang': 'in',
 'place': {'attributes': {},
           'bounding_box': {'coordinates': [[[54.893973, 24.618253],
                                             [54.893973, 25.368672],
                                             [56.208372, 25.368672],
                            

なぜか

coordinates_coordinates, place_bounding_box_coordinatesはlongitude, latitude (経緯度) で

geo_coordinatesはlatitude, longitude (緯経度)

In [6]:
coordinates = geotagged_tweet['place']['bounding_box']['coordinates'][0]

In [7]:
lats = [x[1] for x in coordinates]
longs = [x[0] for x in coordinates]

lats_avg = sum(lats) / len(lats)
longs_avg = sum(longs) / len(longs)

In [8]:
print(longs_avg, lats_avg)
print(geotagged_tweet['coordinates']['coordinates'][0], geotagged_tweet['coordinates']['coordinates'][1])

55.5511725 24.9934625
55.15189319 25.12818472


恐らくcoordinates_coordinates, geo_coordinatesは正確な経緯度で，place_bounding_box_coordinatesはそれが含まれる
区画．単純に長方形で分割しているみたい

quoted_status_id_strが一部欠損している？

In [9]:
for x in tweets:
    if 'quoted_status_id_str' not in x.keys():
        print(x)

{'created_at': 'Tue Jun 06 06:30:00 +0000 2017', 'id_str': '871977455724134400', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'user_id_str': '68872363', 'user_followers_count': 906, 'user_friends_count': 362, 'user_statuses_count': 128183, 'user_lang': 'en', 'user_utc_offset': -14400, 'user_screen_name': 'jessx213', 'text': "RT @iveryy_: I pray to GOD my son ain't like none of you corny niggas 😫", 'in_reply_to_status_id_str': None, 'geo': None, 'coordinates': None, 'place': None, 'retweeted_status_id_str': '871710739433652224', 'entities_hashtags': [], 'entities_urls': [], 'lang': 'en', 'timestamp_ms': '1496730600659', 'emojis': [{'unified': '1F62B', 'short_name': 'tired_face'}]}
{'created_at': 'Tue Jun 06 06:30:00 +0000 2017', 'id_str': '871977455724122112', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'user_id_str': '4516482312', 'user_followers_count': 203, 'user_friends_count': 189

In [28]:
for year in os.listdir(DATA_DIR):
    if year.startswith("."):
        continue
    
    year_path = os.path.join(DATA_DIR, year)    
    for month in os.listdir(year_path):
        if month.startswith("."):
            continue

        month_path = os.path.join(year_path, month)
        for day in os.listdir(month_path):
            if day.startswith("."):
                continue
            
            day_path = os.path.join(month_path, day)
            for hour in os.listdir(day_path):
                if hour.startswith("."):
                    continue
                
                hour_path = os.path.join(day_path, hour)
                for minute in os.listdir(hour_path):
                    if minute.startswith("."):
                        continue

                    minute_path = os.path.join(hour_path, minute)
                    with open(os.path.join(hour_path, "{}{}{}{}{}{}".format(DATA_TYPE, year, month, day, hour, minute)), 'w') as f:
                        
                        subprocess.Popen("cat {} | jq '[.[] | select(.coordinates != null) | {{coordinates, created_at, emojis, place}}]'".format(minute_path), shell=True, stdout=f)
                    
                with open(os.path.join(day_path, '{}{}{}{}{}.json'.format(DATA_TYPE, year, month, day, hour)), 'w') as f:
                    
                    subprocess.Popen("jq -s add {}".format(os.path.join(hour_path, "{}*.json".format(DATA_TYPE))), shell=True, stdout=f)
                    
            with open(os.path.join(month_path, '{}{}{}{}.json'.format(DATA_TYPE, year, month, day)), 'w') as f:
                
                subprocess.Popen("jq -s add {}".format(os.path.join(day_path, "{}*.json".format(DATA_TYPE))), shell=True, stdout=f)
        
        with open(os.path.join(year_path, '{}{}{}.json'.format(DATA_TYPE, year, month)), 'w') as f:
                
            subprocess.Popen("jq -s add {}".format(os.path.join(month_path, "{}*.json".format(DATA_TYPE))), shell=True, stdout=f)
        
    with open(os.path.join(DATA_DIR, '{}{}.json'.format(DATA_TYPE, year)), 'w') as f:
                
        subprocess.Popen("jq -s add {}".format(os.path.join(year_path, "{}*.json".format(DATA_TYPE))), shell=True, stdout=f)